Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [26]:
reg_coeff = 1e-5
num_steps = 10
hl_size = 1024
hl2_size = 300
hl3_size = 50
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    #start with hidden layer random initialization of variables and biases with a low values
    wh1 = weight_variable([image_size*image_size, hl_size])
    bh1 = bias_variable([hl_size])

    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, wh1)+bh1)

    wh2 = weight_variable([hl_size, hl2_size])
    bh2 = bias_variable([hl2_size])

    h2 = tf.nn.relu(tf.matmul(h1, wh2)+bh2)

    keep_prob = tf.placeholder(tf.float32)
    h_drop = tf.nn.dropout(h1, keep_prob)

    # Variables output layer
    wo = weight_variable([hl_size, num_labels])
    bo = bias_variable([num_labels])

    # Training computation output layer
    logits = tf.matmul(h_drop, wo) + bo
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 

    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(wh1) +
                  tf.nn.l2_loss(wo))
    
    # Add the regularization term to the loss.
    loss += reg_coeff * regularizers
  
    # Optimizer. (I let this one here because I have something wrong with AdamOptimizer)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, wh1) + bh1),wo)+bo)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, wh1) + bh1),wo)+bo)

In [27]:
cross_entropy = -tf.reduce_sum(tf_train_labels*tf.log(train_prediction))
#train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(train_prediction,1), tf.argmax(tf_train_labels,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
ct=tf.equal(tf.argmax(train_prediction,1), tf.argmax(tf_train_labels,1))

def runit():
    with tf.Session(graph=graph) as session:
        # This is a one-time operation which ensures the parameters get initialized as
        # we described in the graph: random weights for the matrix, zeros for the
        # biases. 
        tf.initialize_all_variables().run()
        print('Initialized')
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, 
                         tf_train_labels : batch_labels,
                         keep_prob : 0.5}
            _, l, predictions = session.run(
                [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 100 == 0):
                print('Loss at step %d: %f' % (step, l))
                print('Training accuracy: %.1f%%' % accuracy(
                        predictions, train_labels[:len(predictions), :]))
                # Calling .eval() on valid_prediction is basically like calling run(), but
                # just to get that one numpy array. Note that it recomputes all its graph
                # dependencies.
                print('Validation accuracy: %.1f%%' % accuracy(
                        valid_prediction.eval(), valid_labels))

        print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [28]:
num_steps = 3001
hl_size = 1024
batch_size = 128
reg_coeff = 1E-5

runit()

Initialized
Loss at step 0: 5.837964
Training accuracy: 10.2%
Validation accuracy: 27.9%
Loss at step 100: 0.924507
Training accuracy: 5.5%
Validation accuracy: 81.5%
Loss at step 200: 1.176839
Training accuracy: 9.4%
Validation accuracy: 80.8%
Loss at step 300: 0.765717
Training accuracy: 9.4%
Validation accuracy: 82.9%
Loss at step 400: 0.757294
Training accuracy: 9.4%
Validation accuracy: 83.6%
Loss at step 500: 0.679253
Training accuracy: 4.7%
Validation accuracy: 83.8%
Loss at step 600: 0.806305
Training accuracy: 10.9%
Validation accuracy: 84.1%
Loss at step 700: 0.914794
Training accuracy: 10.2%
Validation accuracy: 84.6%
Loss at step 800: 0.564084
Training accuracy: 11.7%
Validation accuracy: 84.7%
Loss at step 900: 0.739369
Training accuracy: 7.8%
Validation accuracy: 84.9%
Loss at step 1000: 0.815553
Training accuracy: 6.2%
Validation accuracy: 84.8%
Loss at step 1100: 0.561938
Training accuracy: 8.6%
Validation accuracy: 85.4%
Loss at step 1200: 0.530172
Training accuracy: 6

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
